<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50 align=left>

---
## Vault Policies
Vault use policies to govern the behavior of lcients and instrument RBAC by specifying autorization.

<img src="images/vault-policy-2.png" width=500>

 
ACL policy **PATH** supports wildcard matching ("**+**", "**\***")  
**PATH** corresponds to Vault API endpoints.  **No policy means no permission**  


List of capabilities  

| List of Permissions | HTTP Verbs |
|------------|------------|
| create | POST/PUT |
| read | GET |
| update | POST/PUT |
| delete | DELETE |
| list | LIST |
| sudo | allows access to paths that are **root-protected** |
| deny | disallows access regardless of other capabilities |

Two out-of-the-box policies, they can't be deleted:
* **root** policy - superuser with all permissions
* **default** policy - common permissions  

**Prerequisites:**
- Vault binary


Before you begin, execute the following command in a separate windows:

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`


## Setup



In [3]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [4]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [ ]:
vault status; vault version

## Create Policy

In [ ]:
# Work with kv secrets engine

cat << EOF > /tmp/static.hcl

path "secret2/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

path "secret1/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

EOF

In [ ]:
cat << EOF > /tmp/static.json
{
"policy": "
   {
    \"path\" : {
           \"secret1/*\" : { \"capabilities\" : [ \"create\", \"read\", \"update\", \"delete\", \"list\"] },
           \"secret2/*\" : { \"capabilities\" : [ \"create\", \"read\", \"update\", \"delete\", \"list\"] }
           }
  }
  "
}
EOF

In [7]:
# Work with kv secrets engine

cat << EOF > /tmp/dynamic.hcl

path "secret1/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

path "mdb1/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

EOF

In [ ]:
cat << EOF > /tmp/sys-policy.json
{
# Configure the ad secrets engine and create roles
path "ad/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

# Mount secrets engines
path "sys/mounts/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

# Write ACL policies
path "sys/policies/acl/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

# Manage tokens for verification
path "auth/token/create" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}

# Manage the leases
path "sys/leases/*" {
  capabilities = [ "create", "read", "update", "delete", "list" ]
}
EOF


**Note:** in *key/value secret engine 2*, the path to write policy would be **kv/data/\<path\>** even though the CLI command to the path is **kv/\<path\>**

### Policy CLI Commands
```
vault policy list|read|write|delete [options]
```

In [ ]:
vault secrets enable -path=secret2/ kv-v2

In [5]:
vault policy write static-policy /tmp/static.hcl

Error opening policy file: open /tmp/static.hcl: no such file or directory


In [ ]:
curl --header "X-Vault-Token: root" --request PUT \
  --data @/tmp/static.json \
  $VAULT_ADDR/v1/sys/policy/staticjs-policy

In [8]:
vault policy write dynamic-policy /tmp/dynamic.hcl

Success! Uploaded policy: dynamic-policy


In [ ]:
vault policy read dynamic-policy

In [ ]:
vault policy list

**Using cURL**

In [ ]:
curl -s --header "X-Vault-Token: $(vault print token)"\
  $VAULT_ADDR/v1/sys/policy | jq

## Associating Policies

During the configuration of users, roles, entities, and groups a list of policies can be specified
```
vault write auth/ldap/groups/sre policies="dev, ops"
```
Another example:
```vault write auth/userpass/users/tbagio password="s3cr3t" policies="base"

In [ ]:
vault write auth/userpass/users/tbagio password="r31nsta11" policies="base"

#### Create a token attached to the newly *base* policy

In [ ]:
vault token create -policy="foo"

**Note:** every token automatically gets *default* policy attached.

In [ ]:
unset VAULT_TOKEN
vault login "s.CO6TDYdpsgE4I4MzmpAvXNf4"

---
## Test "base" Policy

Using base token, you have very limited permissions.

In [ ]:
vault policy list

Using base token, you have very limited permissions.  THe base policy does not have a rule on `sys/policy` path.  Lack of policy means **no permission** on that path.  Therefore, "permission denied" error is the expected behavior.

In [ ]:
vault kv put secret2/training_test password="p@ssw0rd"

#### Read the data back

In [ ]:
vault kv get secret2/training_test

#### Base policy only granst "create" and "read".  With the absence of "update" permission, this operation fails.

In [ ]:
vault kv put secret2/training_test password="password1234"

In [ ]:
vault kv put secret2/team-eng/apikey api_key="123455"

In [ ]:
vault kv delete secret2/team-eng/apikey

In [ ]:
vault kv put secret2/training_ year="2020"

&nbsp;   
### Check Token capabilities
The `vault token capabilities` command fetches the capabilities of a token for a given path.  You can review the policy, but if your token has multiple policies attached, you have to review all the associated policies which can get troublesome.

In [ ]:
vault token capabilities secret2/data/apikey

In [ ]:
vault token capabilities secret2/data/splunk/apikey

#### Try another path that is not permitted by the `base` policy

In [ ]:
vault token capabilities secret2/data/test

In [ ]:
vault login root >/dev/null
vault token capabilities secret2/data/test

---
### Policy for Policies
To allow users to be able to manage policies, you need to write policies for that

In [ ]:
cat << EOF > /tmp/manage-policies.hcl

path "sys/policies/acl/*" {
  capabilities = ["create", "read", "update", "delete", "list", "sudo"]
}
# Create and manage ACL policies via API
path "sys/policies/acl" {
  capabilities = ["list"]
}
EOF

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">